In [ ]:
#spam detection model

In [2]:
import sys
sys.path.append(r'C:\pythonprojects\probability')
sys.path.append(r'C:\pythonprojects\probability\venv\Lib\site-packages')
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab', quiet=True)
import string
df= pd.read_csv('./data/emails.csv')
df.shape
df.text[0]  #checking the email text randomly lets say first

"Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .  we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader : it isguite ciear that  without good products , effective business organization and practicable aim it  will be hotat nowadays market ; but we do promise that your marketing efforts  will become much more effective . here is the list of clear  benefits : creativeness : hand - made , original logos , specially done  to reflect your distinctive company image . convenience : logo and stationery  are provided in all formats ; easy - to - use content management system letsyou  change your website content and even its structure . promptness : you  will see logo drafts within three business days . affordability : your  ma

In [3]:
spam_email_proportion = (df['spam']==1).sum() / df.shape[0] #proportion of spam emails as the spam column with 1 is the spam email
ham_email_proportion = 1-spam_email_proportion   #proportion of ham emails  as the total proportion is 1 so ham emails' proportion can be obtained by subtracting spam proportion from 1
ham_email_proportion
spam_email_proportion  
#print('proportion of spam emails is:', )


np.float64(0.2388268156424581)

In [4]:
df.text[0][9:]  #as the word subject: ends at index 8 , so inorder to remove this word from the email text we are slicing from index 9:

"naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .  we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader : it isguite ciear that  without good products , effective business organization and practicable aim it  will be hotat nowadays market ; but we do promise that your marketing efforts  will become much more effective . here is the list of clear  benefits : creativeness : hand - made , original logos , specially done  to reflect your distinctive company image . convenience : logo and stationery  are provided in all formats ; easy - to - use content management system letsyou  change your website content and even its structure . promptness : you  will see logo drafts within three business days . affordability : your  marketing b

In [5]:
#this function shuffles the emails randomly and removes the word subject using slicing method
#also converts both the text as well as the spam column into numpy array
def preprocess_emails(df):
    df = df.sample(frac = 1 , ignore_index = True, random_state = 42)  #using random_state for consistent random value and frac = 1 means we are taking all the rows of the dataframe and shuffling them
    X = df.text.apply(lambda x: x[9:]).to_numpy()  #slicing from index 9 , and converting into numpy array
    Y = df.spam.to_numpy()  #converting into numpy array
    return X,Y

In [6]:
X,Y = preprocess_emails(df)
X[:2]


array(['re : energy derivatives conference - may 29 , toronto  good morning amy :  vince kaminski will need the following :  an lcd projector to hook up to a lap tap for his presentation  he will have dinner with the conference organizers and speakers on the 29 th .  he will need 2 nights ( the 28 th and the 29 th ) hotel reservations .  he will send you an abstract shortly .  thanks and have a great day !  shirley crenshaw  713 - 853 - 5290  amy aldous on 03 / 31 / 2000 10 : 50 : 11 am  to : shirley . crenshaw @ enron . com  cc :  subject : re : energy derivatives conference - may 29 , toronto  ms . crenshaw ,  thank you for sending the bio so quickly . it \' s exactly what i was looking  for .  we are planning to compile the conference speakers \' papers for distribution  to the participants . while i will not need dr . kaminski \' s contribution for  several weeks , an abstract of his presentation as soon as possible would be  very useful to the conference organizers .  i will also 

In [7]:
email_index = 30
print(X[email_index])  #the index of the text and the spam remains same


karthik rajan - interview schedule  attached you will find the interview packet for the above - referenced person .  the interview will happen friday , march 30 , 2001 . please print all three  documents for your hard copies . if you have any questions , or conflicts of  schedule , please do not hesitate to contact me .  sasha divelbiss  58714


In [19]:
#this function removes the stop words such as and,the,to and also removes the punctuation symbols from the text or email
def preprocess_text(X):
    #here stop is the set of all the stop words such as the , and , is , to and string.punctuation is the punctuation symbols
    stop = set(stopwords.words('english') + list(string.punctuation))
    X_preprocessed = []  #it consists of the cleaned or treated email texts
    if isinstance(X,str):
        X=np.array([X])
    for i,email in enumerate(X):
        #word_tokenize splits the array into each words
        #here we are only appending the words which are not in set called stop and converting the array into the type of original array
        email = np.array([i.lower() for i in word_tokenize(email) if i.lower() not in stop]).astype(X.dtype)  #converting into the same type as that of the passed X
        X_preprocessed.append(email)
    if len(X) == 1:
        return X_preprocessed[0]
    return X_preprocessed    
X_treated = preprocess_text(X)  
email_index = 11
print('Email before treating:',X[email_index])
print('Email after treating:', X_treated[email_index])
X_treated

        
        
        

Email before treating: re : joint probabilities  michael  the updated probabilities are attached . the probability of reaching any fx  times rab multiple are the same as the original analysis . the probabilities  of reaching a given stock price are lower than the original analysis in both  the optimistic and pessimistic cases because the debt levels are higher , and  hence the stock value is lower for any fx - rab value .  bob lee  x 35163
Email after treating: ['joint' 'probabilities' 'michael' 'updated' 'probabilities' 'attached'
 'probability' 'reaching' 'fx' 'times' 'rab' 'multiple' 'original'
 'analysis' 'probabilities' 'reaching' 'given' 'stock' 'price' 'lower'
 'original' 'analysis' 'optimistic' 'pessimistic' 'cases' 'debt' 'levels'
 'higher' 'hence' 'stock' 'value' 'lower' 'fx' 'rab' 'value' 'bob' 'lee'
 'x' '35163']


[array(['energy', 'derivatives', 'conference', 'may', '29', 'toronto',
        'good', 'morning', 'amy', 'vince', 'kaminski', 'need', 'following',
        'lcd', 'projector', 'hook', 'lap', 'tap', 'presentation', 'dinner',
        'conference', 'organizers', 'speakers', '29', 'th', 'need', '2',
        'nights', '28', 'th', '29', 'th', 'hotel', 'reservations', 'send',
        'abstract', 'shortly', 'thanks', 'great', 'day', 'shirley',
        'crenshaw', '713', '853', '5290', 'amy', 'aldous', '03', '31',
        '2000', '10', '50', '11', 'shirley', 'crenshaw', 'enron', 'com',
        'cc', 'subject', 'energy', 'derivatives', 'conference', 'may',
        '29', 'toronto', 'ms', 'crenshaw', 'thank', 'sending', 'bio',
        'quickly', 'exactly', 'looking', 'planning', 'compile',
        'conference', 'speakers', 'papers', 'distribution', 'participants',
        'need', 'dr', 'kaminski', 'contribution', 'several', 'weeks',
        'abstract', 'presentation', 'soon', 'possible', 'would', '

In [20]:
#now we are splitting the text as well as spam into train and test data
# we are training 80 % of data and testing 20 % of data
train_size = int(0.80 * len(X_treated))
train_size  #so upto this index will be our training X and Y which are email text and spam notation
X_trained = X_treated[:train_size]
Y_trained = Y[:train_size]
X_tested = X_treated[train_size:]
Y_tested = Y[train_size:]
print('proportion of spam emails in training dataset is:', (Y_trained == 1).sum() / len(Y_trained))
print('proportion of spam emails in testing dataset is:', (Y_tested == 1).sum() / len(Y_tested))


proportion of spam emails in training dataset is: 0.24312527280663465
proportion of spam emails in testing dataset is: 0.22164048865619546


In [27]:
#which are almost equal, and they are also nearly equal to the proportion of spam emails in the whole data set , so its fine
#now we are implementing the naive's bayes algorithm
def word_frequency(X, Y):
    w_freq = {}
    email_size = len(X)  # total number of emails
    for i in range(email_size):
        spam_denotion = Y[i]  # as the X and Y are of the same index for both email as well as spam denotion
        email = set(X[i])  # here we are using set inorder to prevent the counting of duplicate words from a same email
        for word in email:
            if word not in w_freq.keys():
                w_freq[word] = {'spam': 1,
                                'ham': 1}  # if the current word isnot counted yet then we start initially with value 1 for both spam and ham 
                # which prevents the output to be 0 if the word appears only one time or donot appear in both
            else:
                if spam_denotion == 0:  # if this word is in ham then we increase its count in ham as ham email is denoted by 0 and spam is by 1
                    w_freq[word]['ham'] += 1
                else:
                    w_freq[word]['spam'] += 1

    return w_freq
word_frequency= word_frequency(X_trained,Y_trained)    
word_frequency                    
                    

{'need': {'spam': 182, 'ham': 773},
 'joining': {'spam': 2, 'ham': 66},
 'help': {'spam': 129, 'ham': 556},
 'much': {'spam': 100, 'ham': 442},
 '28': {'spam': 14, 'ham': 297},
 'lap': {'spam': 3, 'ham': 5},
 'requirements': {'spam': 21, 'ham': 90},
 'see': {'spam': 214, 'ham': 650},
 'sending': {'spam': 33, 'ham': 161},
 '30': {'spam': 90, 'ham': 589},
 'vincent': {'spam': 2, 'ham': 121},
 'compile': {'spam': 1, 'ham': 8},
 'best': {'spam': 254, 'ham': 569},
 'cc': {'spam': 5, 'ham': 1718},
 'aldous': {'spam': 1, 'ham': 4},
 '853': {'spam': 2, 'ham': 302},
 'information': {'spam': 270, 'ham': 722},
 'know': {'spam': 152, 'ham': 1229},
 'ordinator': {'spam': 1, 'ham': 11},
 'quickly': {'spam': 30, 'ham': 67},
 'staying': {'spam': 2, 'ham': 28},
 'great': {'spam': 113, 'ham': 357},
 'also': {'spam': 129, 'ham': 893},
 '10': {'spam': 181, 'ham': 952},
 '5290': {'spam': 1, 'ham': 145},
 'crenshaw': {'spam': 1, 'ham': 428},
 'fax': {'spam': 34, 'ham': 514},
 'projector': {'spam': 2, 'ham':

In [22]:
#here we are counting the total number of spam emails as well as ham emails or non_spam emails
#and storing them in their respective class key
class_frequency={'spam':(Y_trained==1).sum(),'ham':(Y_trained==0).sum()} 
class_frequency
total_emails = len(Y_trained)
total_emails

4582

In [30]:
#this function calculates the propbability of word appearing in a given class
def prob_word_given_class(word,cls,word_frequency,class_frequency):
    if word not in word_frequency:  #if the word in the testing dataset is not in the word freq which is made using training dataset , then we just return 1
        return 1
    amount_words_class = word_frequency[word][cls]  #total number of words under the given class
    total_class_emails = class_frequency[cls]  #total number of emails under the given class
    prob = amount_words_class / total_class_emails
    return prob
print(f"P(lottery | spam) = {prob_word_given_class('lottery', cls = 'spam', word_frequency = word_frequency, class_frequency = class_frequency)}")
print(f"P(lottery | ham) = {prob_word_given_class('lottery', cls = 'ham', word_frequency = word_frequency, class_frequency = class_frequency)}")
print(f"P(schedule | spam) = {prob_word_given_class('schedule', cls = 'spam', word_frequency = word_frequency, class_frequency = class_frequency)}")
print(f"P(schedule | ham) = {prob_word_given_class('schedule', cls = 'ham', word_frequency = word_frequency, class_frequency = class_frequency)}")    
    

P(lottery | spam) = 0.00718132854578097
P(lottery | ham) = 0.0002883506343713956
P(schedule | spam) = 0.008976660682226212
P(schedule | ham) = 0.10265282583621683


In [31]:
def prob_email_given_class(treated_email,cls,word_frequency,class_frequency):
    p= 1 #initial product of probability
    for word in treated_email:
        p*=prob_word_given_class(word,cls,word_frequency,class_frequency)
    return p  #p is the probability of email given class
example_email = "Click here to win a lottery ticket and claim your prize!"
treated_email = preprocess_text(example_email)
prob_spam = prob_email_given_class(treated_email, cls = 'spam', word_frequency = word_frequency, class_frequency = class_frequency) #probability based on class spam
prob_ham = prob_email_given_class(treated_email, cls = 'ham', word_frequency = word_frequency, class_frequency = class_frequency)  #based on ham
print(f"Email: {example_email}\nEmail after preprocessing: {treated_email}\nP(email | spam) = {prob_spam}\nP(email | ham) = {prob_ham}")     
        

Email: Click here to win a lottery ticket and claim your prize!
Email after preprocessing: ['click' 'win' 'lottery' 'ticket' 'claim' 'prize']
P(email | spam) = 4.117424263085333e-11
P(email | ham) = 1.1298495335380886e-15


In [32]:
def naive_bayes(treated_email,word_frequency,class_frequency,return_likelihood=False):  #our naive bayes pipeline for detecting whether the given email is spam or not
    prob_email_given_spam = prob_email_given_class(treated_email,'spam',word_frequency,class_frequency)
    prob_email_given_ham = prob_email_given_class(treated_email,'ham',word_frequency,class_frequency)
    prob_spam = class_frequency['spam'] / total_emails
    prob_ham = class_frequency['ham'] / total_emails
    spam_likelihood = prob_spam * prob_email_given_spam  #probability of an email being spam
    ham_likelihood = prob_ham * prob_email_given_ham
    if return_likelihood:
        return (spam_likelihood,ham_likelihood)
    if spam_likelihood > ham_likelihood:  #if the probability of being spam is greater than ham then the given mail is spam mail so we return 1 which denotes being spam
        return 1
    else:
        return 0
example_email = "Click here to win a lottery ticket and claim your prize!"
treated_email = preprocess_text(example_email)

print(f"Email: {example_email}\nEmail after preprocessing: {treated_email}\nNaive Bayes predicts this email as: {naive_bayes(treated_email, word_frequency, class_frequency)}")

print("\n\n")
example_email = "Our meeting will happen in the main office. Please be there in time."
treated_email = preprocess_text(example_email)

print(f"Email: {example_email}\nEmail after preprocessing: {treated_email}\nNaive Bayes predicts this email as: {naive_bayes(treated_email, word_frequency, class_frequency)}")        
    
    

Email: Click here to win a lottery ticket and claim your prize!
Email after preprocessing: ['click' 'win' 'lottery' 'ticket' 'claim' 'prize']
Naive Bayes predicts this email as: 1



Email: Our meeting will happen in the main office. Please be there in time.
Email after preprocessing: ['meeting' 'happen' 'main' 'office' 'please' 'time']
Naive Bayes predicts this email as: 0


In [ ]:
#here we are testing our model using the training dataset
def test_model():
    Y_pred = []
    for email in X_tested:
     prediction = naive_bayes(email,word_frequency,class_frequency,return_likelihood=False)  #prediction done through our model naive bayes pipeline
     Y_pred.append(prediction) #either gets 0 for ham or 1 for spam
    return Y_pred
Y_pred =  test_model()   

In [42]:
#calculation of the accuracy , we are checking how accurately our model predicts the email being spam or not spam
#inorder to calculate the accuracy we need to have true positives and true negatives 
#and for getting this value , we must work with the testing datasets not training datasets
#training dataset is used for model training 
#testing dataset is used for testing our model
def get_positives(Y_test,Y_pred):  
    if len(Y_test) !=len(Y_pred):
        return('The length of the Y_true label must match with the Y_predicted label')
    true_positives = 0
    for i in range(len(Y_test)):
        if Y_test[i] == 1 and Y_pred[i] == 1:  #comparing the predicted value and the true value 
            true_positives+=1
    return true_positives   
def get_negatives(Y_test,Y_pred):
    if len(Y_test) !=len(Y_pred):
        return('The length of the Y_true label must match with the Y_predicted label')
    true_negatives = 0
    for i in range(len(Y_test)):
        if Y_test[i] == 0 and Y_pred[i] == 0:  #comparing the predicted value and the true value 
            true_negatives+=1
    return true_negatives    
    

true_positives = get_positives(Y_tested,Y_pred)
true_negatives = get_negatives(Y_tested,Y_pred)
accuracy = (true_positives + true_negatives) / len(Y_pred)
print('The percentage by which our model predicts the email being spam or not is:', accuracy*100)

The percentage by which our model predicts the email being spam or not is: 95.98603839441536


In [47]:
example_index = 4798
example_email = X[example_index]
treated_email = preprocess_text(example_email)
print(f"The email is:\n\t{example_email}\n\nAfter preprocessing:\n\t:{treated_email}")

The email is:
	from the enron india newsdesk - may 5 - 7 newsclips  stinson / vince ,  some news articles . do read the first one , and the second last one .  regards ,  sandeep .  - - - - - - - - - - - - - - - - - - - - - - forwarded by sandeep kohli / enron _ development on  05 / 07 / 2001 09 : 10 am - - - - - - - - - - - - - - - - - - - - - - - - - - -  nikita varma  05 / 07 / 2001 07 : 42 am  to : nikita varma / enron _ development @ enron _ development  cc : ( bcc : sandeep kohli / enron _ development )  subject : from the enron india newsdesk - may 5 - 7 newsclips  the economic times , may 7 , 2001  enron ceo casts vote to save dpc , tina edwin & soma banerjee  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  the economic times , may 7 , 2001  maha sore over delay in naming godbole nominee  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  the times of india , 7 may , 2001  maharashtra ' unhappy ' with delay in naming godbole nominee  - - - 

In [48]:
spam_likelihood,ham_likelihood = naive_bayes(treated_email,word_frequency,class_frequency,return_likelihood=True)
spam_likelihood,ham_likelihood
#here we are getting 0 cause the probability of this example email under both class is way too small which the PCs couldnot handle and regarder them as 0
#which is also known as underflow problem
#and the probability must not be 0

(np.float64(0.0), np.float64(0.0))

In [50]:
#so to prevent this issue we must use log property which turns the product into summation that stabilizes the operation of extremely small numbers
def log_prob_email_given_class(treated_email,cls,word_frequency,class_frequency):
    p= 0 #initial product of probability
    for word in treated_email:
        if word in word_frequency:  #we are doing the probability for word only if the word is in wordfreq otherwise we might face problem of missing word in word_freq
            p+=np.log(prob_word_given_class(word,cls,word_frequency,class_frequency))  #using log
    return p  #p is the probability of email given class
    

In [59]:
def log_naive_bayes(treated_email,word_frequency,class_frequency,return_likelihood=False):  #our naive bayes pipeline for detecting whether the given email is spam or not
    prob_email_given_spam = log_prob_email_given_class(treated_email,'spam',word_frequency,class_frequency)
    prob_email_given_ham = log_prob_email_given_class(treated_email,'ham',word_frequency,class_frequency)
    prob_spam = class_frequency['spam'] / total_emails
    prob_ham = class_frequency['ham'] / total_emails
    spam_likelihood = np.log(prob_spam) + prob_email_given_spam  #probability of an email being spam
    ham_likelihood = np.log(prob_ham) + prob_email_given_ham
    if return_likelihood:
        return (spam_likelihood,ham_likelihood)
    if spam_likelihood > ham_likelihood:  #if the probability of being spam is greater than ham then the given mail is spam mail so we return 1 which denotes being spam
        return 1
    else:
        return 0

In [60]:
prediction_example = log_naive_bayes(treated_email,word_frequency,class_frequency,return_likelihood=False)
print('Prediction of example email using log naive bayes method is:', prediction_example)


Prediction of example email using log naive bayes method is: 0


In [61]:
#now calculating the accuracy but with logged pipeline
Y_pred_logged = []
for email in X_tested:
    prediction = log_naive_bayes(email,word_frequency,class_frequency,return_likelihood=False)
    Y_pred_logged.append(prediction)
logged_true_positives = get_positives(Y_tested,Y_pred_logged)
logged_true_negatives = get_negatives(Y_tested,Y_pred_logged)    
logged_accuracy = (logged_true_positives+logged_true_negatives)/len(Y_tested)
logged_accuracy
#which makes our model way more accurate close to 100%    

0.9904013961605584